# Getting Started

In [2]:
## SORA package
from sora import Occultation, Body, Star, LightCurve, Observer, EphemHorizons
from sora.prediction import prediction, occ_params
from sora.extra import draw_ellipse

## Other main packages
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u

## Usual packages
import numpy as np
import matplotlib.pylab as pl
import os
import sys

# Add the directory to sys.path
sys.path.append('/home/joshuabartkoske/AO/scripts')
from Occultations2 import Open_Two_Column_File

SORA version: 0.3.1


In [3]:
# First, define the Solar System body that was in the Dyachenko 2021 paper
sylvia = Body(name="Sylvia")

Obtaining data for Sylvia from SBDB


In [4]:
number = int(sylvia.shortname.split()[0])
print(number)

87


In [5]:
sylvia_pred     = prediction(body=sylvia, time_beg='2019-12-11',time_end='2019-12-13',mag_lim=12)

Ephemeris was split in 1 parts for better search of stars

Searching occultations in part 1/1
Generating Ephemeris between 2019-12-11 00:00:00.000 and 2019-12-12 23:59:00.000 ...
    3 GaiaDR3 stars downloaded
Identifying occultations ...

1 occultations found.


        Use get_body("moon") instead. [sora.prediction.table]


In [6]:
print(sylvia_pred.colnames)
print(sylvia_pred.dtype)

['Epoch', 'ICRS Star Coord at Epoch', 'Geocentric Object Position', 'C/A', 'P/A', 'Vel', 'Dist', 'G', 'long', 'loct', 'M-G-T', 'S-G-T', 'GaiaDR3 Source ID']
[('Epoch', 'O'), ('ICRS Star Coord at Epoch', 'O'), ('Geocentric Object Position', 'O'), ('C/A', '<f8'), ('P/A', '<f8'), ('Vel', '<f8'), ('Dist', '<f8'), ('G', '<f8'), ('long', '<f8'), ('loct', '<U5'), ('M-G-T', '<f8'), ('S-G-T', '<f8'), ('GaiaDR3 Source ID', '<U18')]


In [7]:
bettina = Body(name='Bettina')
bettina_pred  = prediction(body=bettina, time_beg='2020-12-23',time_end='2020-12-25',mag_lim=14)
bettina_pred["2020-12-24 10:15:55.860"]

Obtaining data for Bettina from SBDB
Ephemeris was split in 1 parts for better search of stars

Searching occultations in part 1/1
Generating Ephemeris between 2020-12-23 00:00:00.000 and 2020-12-24 23:59:00.000 ...
    6 GaiaDR3 stars downloaded
Identifying occultations ...

1 occultations found.


        Use get_body("moon") instead. [sora.prediction.table]


Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str18
2020-12-24 10:15:55.860,04 00 37.75735 +35 52 56.50355,04 00 37.74891 +35 52 56.96813,0.476,347.55,-8.46,1.856,10.350,173,21:47,40,149,219334294660438528


In [8]:
print(bettina_pred.colnames)
print(bettina_pred["Epoch"])
bettina_epoch = Time(bettina_pred["Epoch"][0])
print(bettina_epoch)

['Epoch', 'ICRS Star Coord at Epoch', 'Geocentric Object Position', 'C/A', 'P/A', 'Vel', 'Dist', 'G', 'long', 'loct', 'M-G-T', 'S-G-T', 'GaiaDR3 Source ID']
         Epoch         
-----------------------
2020-12-24 10:15:55.860
2020-12-24 10:15:55.860


In [9]:
sylvia_number = int(sylvia.shortname.split()[0])
sylvia_pred.add_column(name="MPC Number", col=[sylvia_number])

In [10]:
sylvia_pred

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID,MPC Number
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str18,int64
2019-12-12 01:39:36.060,08 25 36.26861 +28 08 29.38428,08 25 36.32909 +28 08 30.18678,1.133,44.91,-9.35,2.918,10.649,21,03:03,45,137,707188325455532416,87


In [11]:
bettina_number = int(bettina.shortname.split()[0])
bettina_pred.add_column(name="MPC Number", col=[bettina_number])
sylvia_pred.add_row([bettina_pred[col] for col in sylvia_pred.colnames])

In [12]:
sylvia_pred['ICRS Star Coord at Epoch'] = SkyCoord(sylvia_pred['ICRS Star Coord at Epoch'], unit=(u.hourangle, u.deg))
sylvia_pred['Geocentric Object Position'] = SkyCoord(sylvia_pred['Geocentric Object Position'], unit=(u.hourangle, u.deg))
sylvia_pred["Epoch"][1] = bettina_epoch
print(sylvia_pred["Epoch"])

         Epoch         
-----------------------
2019-12-12 01:39:36.060
2020-12-24 10:15:55.860


In [13]:
sylvia_pred

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID,MPC Number
,"deg,deg","deg,deg,AU",arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,,
object,SkyCoord,SkyCoord,float64,float64,float64,float64,float64,float64,str5,float64,float64,str18,int64
2019-12-12 01:39:36.060,08 25 36.26861 +28 08 29.38428,08 25 36.32909 +28 08 30.18678,1.133,44.91,-9.35,2.918,10.649,21,03:03,45,137,707188325455532416,87
2020-12-24 10:15:55.860,04 00 37.75735 +35 52 56.50355,04 00 37.74891 +35 52 56.96813,0.476,347.55,-8.46,1.856,10.350,173,21:47,40,149,219334294660438528,250


In [14]:
new_sylvia = Body(name=sylvia_pred[0]["MPC Body Number"])

KeyError: 'MPC Body Number'

In [ ]:
# Define BTA as a location: We will use T1 as our coordinates:
BTA = Observer(name="Zelenchuksaya", code="115")

In [ ]:
star_occ = Star(code='707188325455532416')

print(star_occ)

1 GaiaDR3 star found band={'G': 10.648536}
star coordinate at J2016.0: RA=8h25m36.27821s +/- 0.0216 mas, DEC=28d08m29.4526s +/- 0.0181 mas

GaiaDR3 star Source ID: 707188325455532416
ICRS star coordinate at J2016.0:
RA=8h25m36.27821s +/- 0.0216 mas, DEC=28d08m29.4526s +/- 0.0181 mas
pmRA=-33.157 +/- 0.026 mas/yr, pmDEC=-17.278 +/- 0.018 mas/yr
GaiaDR3 Proper motion corrected as suggested by Cantat-Gaudin & Brandt (2021) 
Plx=5.7696 +/- 0.0297 mas, Rad. Vel.=32.22 +/- 0.42 km/s 

Magnitudes: G: 10.649, B: 11.304, V: 10.942, R: 10.700, J:  9.527, H:  9.190,
            K:  9.092

Apparent diameter from Kervella et. al (2004):
    V: 0.0689 mas, B: 0.0654 mas
Apparent diameter from van Belle (1999):
    sg: B: 0.0748 mas, V: 0.0782 mas
    ms: B: 0.0760 mas, V: 0.0631 mas
    vs: B: 0.1112 mas, V: 0.1009 mas


In [ ]:
occ = Occultation(star=star_occ, body=sylvia, time='2019-12-12 01:39:36.060')

print(occ)

        Use get_body("moon") instead. [sora.prediction.table]


Stellar occultation of star GaiaDR3 707188325455532416 by 87 Sylvia (A866 KA).

Geocentric Closest Approach: 1.133 arcsec
Instant of CA: 2019-12-12 01:39:36.040
Position Angle: 44.90 deg
Geocentric shadow velocity: -9.35 km / s
Sun-Geocenter-Target angle:  136.77 deg
Moon-Geocenter-Target angle: 45.42 deg


No observations reported

###############################################################################
                                     STAR                                      
###############################################################################
GaiaDR3 star Source ID: 707188325455532416
ICRS star coordinate at J2016.0:
RA=8h25m36.27821s +/- 0.0216 mas, DEC=28d08m29.4526s +/- 0.0181 mas
pmRA=-33.157 +/- 0.026 mas/yr, pmDEC=-17.278 +/- 0.018 mas/yr
GaiaDR3 Proper motion corrected as suggested by Cantat-Gaudin & Brandt (2021) 
Plx=5.7696 +/- 0.0297 mas, Rad. Vel.=32.22 +/- 0.42 km/s 

Magnitudes: G: 10.649, B: 11.304, V: 10.942, R: 10.700, J:  9.527, H:  9.190,
   

**Double Check for the Velocity of the occultation shadow**

This uses sora.occultation.utils.positionv(star, ephem, observer, time), so we need the star as a Star object, the ephemeris as an Ephemeris object, the observer as an Observer object, and the time as an astropy Time object.

In [ ]:
sylvia_ephem = EphemHorizons(name="Sylvia")

In [ ]:
sylvia_occ_time = Time('2019-12-12 01:39:36.060')

In [ ]:
from sora.occultation.utils import positionv
EW_pos, NS_pos, EW_vel, NS_vel = positionv(star=star_occ, ephem=sylvia_ephem, observer=BTA, time=sylvia_occ_time)
# my one issue is I don't know the units on this but I am guessing kilometers?
# there are also no units in the documentation of this function.
print(EW_pos)

/home/joshuabartkoske/mySORA_env/lib/python3.10/site-packages/sora/occultation/utils.py:52: FutureWarning: geocentric is deprecated and will be removed in v1.0; Please use get_position(time=time, observer='geocenter')
  coord = star.geocentric(time)


-105.8534489503586


In [ ]:
# now calculate the full velocity magnitude
Occ_shadow_vel_magnitude = np.sqrt(EW_vel**2 + NS_vel**2)
print(f"EW velocity: {EW_vel}")
print(f"NS velocity: {NS_vel}")
print(f"|velocity| : {Occ_shadow_vel_magnitude}")


EW velocity: 6.900873759313981
NS velocity: -6.573646343435939
|velocity| : 9.530733701586515


The velocity values do indeed agree with the paper from 2021. I consider this to be final proof for using this velocity value for all future and current analyses of the occultations we currently have in the pipeline.

**VERITAS Occultation Velocities**

Now, we will go through all of the VERITAS occultation and verify their velocities and use the values produced by this SORA prediction software

In [15]:
# Define VERITAS as a location: We will use T1 as our coordinates:
VERITAS = Observer(name="VERITAS", lon='-110 57 3.456', lat='31 40 29.5572', height =1268)

In [16]:
# define all of the bodies
imprinetta = Body(name='Imprinetta')
severny = Body(name='Severny')
yrsa = Body(name='Yrsa')
egeria = Body(name='Egeria')
bettina = Body(name='Bettina')
polyxo = Body(name='Polyxo')
lehigh = Body(name='Lehigh')
sternberga = Body(name='Sternberga')

Obtaining data for Imprinetta from SBDB
Obtaining data for Severny from SBDB
Obtaining data for Yrsa from SBDB
Obtaining data for Egeria from SBDB
Obtaining data for Bettina from SBDB
Obtaining data for Polyxo from SBDB
Obtaining data for Lehigh from SBDB
Obtaining data for Sternberga from SBDB


In [17]:
# get all of the occultations VERITAS observed
imprinetta_pred     = prediction(body=imprinetta, time_beg='2018-02-21',time_end='2018-02-23',mag_lim=14)
severny_pred        = prediction(body=severny, time_beg='2019-10-20',time_end='2019-10-22',mag_lim=14)
yrsa_pred           = prediction(body=yrsa, time_beg='2019-10-31',time_end='2019-11-03',mag_lim=14)
egeria_pred         = prediction(body=egeria, time_beg='2020-12-18',time_end='2020-12-20',mag_lim=14)
bettina_pred        = prediction(body=bettina, time_beg='2020-12-23',time_end='2020-12-25',mag_lim=14)
polyxo_pred         = prediction(body=polyxo, time_beg='2021-12-04',time_end='2021-12-06',mag_lim=14)
lehigh_pred         = prediction(body=lehigh, time_beg='2022-06-02',time_end='2022-06-04',mag_lim=14)
sternberga_pred     = prediction(body=sternberga, time_beg='2023-05-13',time_end='2023-05-15',mag_lim=14)

Ephemeris was split in 1 parts for better search of stars

Searching occultations in part 1/1
Generating Ephemeris between 2018-02-21 00:00:00.000 and 2018-02-22 23:59:00.000 ...
    4 GaiaDR3 stars downloaded
Identifying occultations ...


        Use get_body("moon") instead. [sora.prediction.table]



1 occultations found.
Ephemeris was split in 1 parts for better search of stars

Searching occultations in part 1/1
Generating Ephemeris between 2019-10-20 00:00:00.000 and 2019-10-21 23:59:00.000 ...
    5 GaiaDR3 stars downloaded
Identifying occultations ...


        Use get_body("moon") instead. [sora.prediction.table]



1 occultations found.
Ephemeris was split in 1 parts for better search of stars

Searching occultations in part 1/1
Generating Ephemeris between 2019-10-31 00:00:00.000 and 2019-11-02 23:59:00.000 ...
    55 GaiaDR3 stars downloaded
Identifying occultations ...


...

        Use get_body("moon") instead. [sora.prediction.table]



1 occultations found.
Ephemeris was split in 1 parts for better search of stars

Searching occultations in part 1/1
Generating Ephemeris between 2022-06-02 00:00:00.000 and 2022-06-03 23:59:00.000 ...
    12 GaiaDR3 stars downloaded
Identifying occultations ...


        Use get_body("moon") instead. [sora.prediction.table]



2 occultations found.
Ephemeris was split in 1 parts for better search of stars

Searching occultations in part 1/1
Generating Ephemeris between 2023-05-13 00:00:00.000 and 2023-05-14 23:59:00.000 ...
    2 GaiaDR3 stars downloaded
Identifying occultations ...

1 occultations found.


        Use get_body("moon") instead. [sora.prediction.table]


In [18]:
# select the correct occultations from each event
imprinetta_pred["2018-02-22 11:24:54.660"]
severny_pred['2019-10-21 09:35:39.820']
yrsa_pred["2019-11-01 02:26:22.540"]
egeria_pred["2020-12-19 06:13:50.680"]
bettina_pred["2020-12-24 10:15:55.860"]
polyxo_pred["2021-12-05 08:12:55.880"]
lehigh_pred["2022-06-03 09:18:40.54"]
sternberga_pred["2023-05-14 04:41:56.500"]

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str18
2023-05-14 04:41:56.500,08 48 41.90350 +05 07 41.58695,08 48 41.90031 +05 07 42.90090,1.315,357.92,20.51,3.041,11.579,190,17:21,145,81,582680075402278528


In [ ]:
# Bettina parameters
bettina_pred["2020-12-24 10:15:55.860"]

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str18
2020-12-24 10:15:55.860,04 00 37.75735 +35 52 56.50355,04 00 37.74891 +35 52 56.96813,0.476,347.55,-8.46,1.856,10.350,173,21:47,40,149,219334294660438528


In [ ]:
# Egeria parameters
egeria_pred["2020-12-19 06:13:50.680"]

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str18
2020-12-19 06:13:50.680,05 55 35.54739 +43 26 22.68355,05 55 35.49285 +43 26 21.14913,1.645,201.16,-11.52,1.440,11.189,267,00:02,123,160,193323800915713024


In [ ]:
bettina_pred["2020-12-24 10:15:55.860"].plot_occ_map(nameimg='figures/predictions/Bettina_20201224_map')

figures/predictions/Bettina_20201224_map.png generated


In [ ]:
# calculated the exact occultation values at the time and location of the occultation
bettina_time = Time("2020-12-24 10:15:55.860")
bettina_ephem = EphemHorizons(name='Bettina')
bettina_star = Star(catalogue='gaiadr3', code='219334294660438528')

1 GaiaDR3 star found band={'G': 10.350464}
star coordinate at J2016.0: RA=4h00m37.76061s +/- 0.0189 mas, DEC=35d52m56.5507s +/- 0.0127 mas



In [ ]:
time_of_closest_approach, distance_of_closest_approach, position_angle_of_closest_approach, velocity_shadow, distance_to_object = occ_params(star=bettina_star, ephem=bettina_ephem, time=bettina_time, max_tdiff=600, reference_center=VERITAS)

print("Occultation of Bettina by TYC 2369-02140-1 over VERITAS")
print("--------------------------------------------------------")
print(f"Time of closest approach: {time_of_closest_approach}")
print(f"Distance of closest approach: {distance_of_closest_approach} ")
print(f"Position angle of closest approach: {position_angle_of_closest_approach}")
print(f"Velocity of occultation shadow: {velocity_shadow} ")
print(f"Distance to the occulting body: {distance_to_object.to(u.km)}")


Occultation of Bettina by TYC 2369-02140-1 over VERITAS
--------------------------------------------------------
Time of closest approach: 2020-12-24 10:05:09.900
Distance of closest approach: 0.011647189748489786 arcsec 
Position angle of closest approach: 346.3017259953651 deg
Velocity of occultation shadow: -8.615259151767532 km / s 
Distance to the occulting body: 277718113.42636544 km


In [ ]:
severny_pred['2019-10-21 09:35:39.820']

NameError: name 'severny_pred' is not defined

In [ ]:
lehigh_pred["2022-06-03 09:18:40.54"]

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str19
2022-06-03 09:18:40.540,16 58 20.27066 -18 10 35.08829,16 58 20.24743 -18 10 32.45048,2.658,352.85,-14.36,2.253,12.139,223,00:11,139,175,4129613739470860928


In [20]:
# calculated the exact occultation values at the time and location of the occultation
lehigh_time = Time("2022-06-03 09:18:40.540")
lehigh_ephem = EphemHorizons(name='Lehigh')
lehigh_star = Star(catalogue='gaiadr3', code='4129613739470860928')

1 GaiaDR3 star found band={'G': 12.138798}
star coordinate at J2016.0: RA=16h58m20.27341s +/- 0.0138 mas, DEC=-18d10m35.0601s +/- 0.0094 mas

Magnitudes in ['V'] were not located in NOMAD


In [21]:
time_of_closest_approach, distance_of_closest_approach, position_angle_of_closest_approach, velocity_shadow, distance_to_object = occ_params(star=lehigh_star, ephem=lehigh_ephem, time=lehigh_time, max_tdiff=600, reference_center=VERITAS)

print("Occultation of Lehigh over VERITAS")
print("--------------------------------------------------------")
print(f"Time of closest approach: {time_of_closest_approach}")
print(f"Distance of closest approach: {distance_of_closest_approach} ")
print(f"Position angle of closest approach: {position_angle_of_closest_approach}")
print(f"Velocity of occultation shadow: {velocity_shadow} ")
print(f"Distance to the occulting body: {distance_to_object.to(u.km)}")

Occultation of Lehigh over VERITAS
--------------------------------------------------------
Time of closest approach: 2022-06-03 09:15:21.640
Distance of closest approach: 0.02133962452091488 arcsec 
Position angle of closest approach: 173.12515885407635 deg
Velocity of occultation shadow: -14.706341962985057 km / s 
Distance to the occulting body: 337034001.5400557 km


In [22]:
polyxo_pred["2021-12-05 08:12:55.880"]

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str19
2021-12-05 08:12:55.880,04 48 46.84112 +16 02 46.90153,04 48 46.82895 +16 02 48.05239,1.164,351.33,-12.80,1.847,11.410,235,23:51,163,174,3405131371706544256


In [23]:
# calculated the exact occultation values at the time and location of the occultation
polyxo_time = Time("2021-12-05 08:12:55.880")
polyxo_ephem = EphemHorizons(name='Polyxo')
polyxo_star = Star(catalogue='gaiadr3', code='3405131371706544256')

1 GaiaDR3 star found band={'G': 11.41042}
star coordinate at J2016.0: RA=4h48m46.83985s +/- 0.0182 mas, DEC=16d02m46.9350s +/- 0.0095 mas

Magnitudes in ['V'] were not located in NOMAD


In [24]:
time_of_closest_approach, distance_of_closest_approach, position_angle_of_closest_approach, velocity_shadow, distance_to_object = occ_params(star=polyxo_star, ephem=polyxo_ephem, time=polyxo_time, max_tdiff=600, reference_center=VERITAS)

print("Occultation of Polyxo over VERITAS")
print("--------------------------------------------------------")
print(f"Time of closest approach: {time_of_closest_approach}")
print(f"Distance of closest approach: {distance_of_closest_approach} ")
print(f"Position angle of closest approach: {position_angle_of_closest_approach}")
print(f"Velocity of occultation shadow: {velocity_shadow} ")
print(f"Distance to the occulting body: {distance_to_object.to(u.km)}")

Occultation of Polyxo over VERITAS
--------------------------------------------------------
Time of closest approach: 2021-12-05 08:10:56.760
Distance of closest approach: 0.024058667588989766 arcsec 
Position angle of closest approach: 351.4619246645195 deg
Velocity of occultation shadow: -13.18388502091863 km / s 
Distance to the occulting body: 276320045.7680598 km


In [25]:
yrsa_pred["2019-11-01 02:26:22.540"]

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str19
2019-11-01 02:26:22.540,19 24 55.91120 -26 01 31.03889,19 25 19.84353 -26 00 49.14284,1.848,352.94,24.76,3.374,9.469,215,16:44,20,71,6766466231748780160


In [26]:
# calculated the exact occultation values at the time and location of the occultation
yrsa_time = Time("2019-11-01 02:26:22.540")
yrsa_ephem = EphemHorizons(name='Yrsa')
yrsa_star = Star(catalogue='gaiadr3', code='6766466231748780160')

1 GaiaDR3 star found band={'G': 9.468946}
star coordinate at J2016.0: RA=19h24m55.91153s +/- 0.0178 mas, DEC=-26d01m31.0263s +/- 0.0164 mas



In [27]:
time_of_closest_approach, distance_of_closest_approach, position_angle_of_closest_approach, velocity_shadow, distance_to_object = occ_params(star=yrsa_star, ephem=yrsa_ephem, time=yrsa_time, max_tdiff=600, reference_center=VERITAS)

print("Occultation of Yrsa over VERITAS")
print("--------------------------------------------------------")
print(f"Time of closest approach: {time_of_closest_approach}")
print(f"Distance of closest approach: {distance_of_closest_approach} ")
print(f"Position angle of closest approach: {position_angle_of_closest_approach}")
print(f"Velocity of occultation shadow: {velocity_shadow} ")
print(f"Distance to the occulting body: {distance_to_object.to(u.km)}")

Occultation of Yrsa over VERITAS
--------------------------------------------------------
Time of closest approach: 2019-11-01 02:28:51.840
Distance of closest approach: 0.0005029288022564492 arcsec 
Position angle of closest approach: 183.57748030628323 deg
Velocity of occultation shadow: 24.451863114986562 km / s 
Distance to the occulting body: 504775418.55482376 km
